In [1]:
!pip install arxiv transformers datasets accelerate bitsandbytes peft unsloth
!pip install auto-gptq optimum xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.6/191.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 102.2 MB/s eta 0:00:00


In [1]:
import arxiv
import json

# Fetch recent AI research papers
search_query = "Artificial Intelligence"
client = arxiv.Client()
search = arxiv.Search(
    query=search_query,
    max_results=50,
    sort_by=arxiv.SortCriterion.SubmittedDate
)

papers = []
for result in client.results(search):
    papers.append({
        "title": result.title,
        "abstract": result.summary,
        "url": result.entry_id
    })

# Save dataset
with open("ai_papers.json", "w") as f:
    json.dump(papers, f, indent=4)

print(f"Collected {len(papers)} AI research papers.")


Collected 50 AI research papers.


In [2]:
from transformers import pipeline
import json

# Load AI research papers
with open("ai_papers.json", "r") as f:
    papers = json.load(f)

qa_pipeline = pipeline("text2text-generation", model="t5-small")

qa_dataset = []
for paper in papers:
    abstract = paper["abstract"]
    qna = qa_pipeline(f"Generate question-answer pairs from: {abstract}", max_length=256)
    qa_dataset.append({"context": abstract, "qa": qna[0]['generated_text']})

# Save dataset
with open("qa_dataset.json", "w") as f:
    json.dump(qa_dataset, f, indent=4)

print("QA dataset generated successfully.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


QA dataset generated successfully.


In [3]:
%%capture
!pip install transformers accelerate datasets peft bitsandbytes

In [4]:
%%capture
!pip install transformers accelerate datasets peft bitsandbytes llama-cpp-python

In [7]:


from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
import torch
import json
import random
from peft import LoraConfig, get_peft_model

# Load Qwen model and tokenizer
model_name = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True, device_map="auto")

# Enable gradient checkpointing to reduce memory usage
model.gradient_checkpointing_enable()

# Apply LoRA configuration for fine-tuning
lora_config = LoraConfig(
    r=8, lora_alpha=16, target_modules=["q_proj", "v_proj"], lora_dropout=0.1, bias="none"
)
model = get_peft_model(model, lora_config)

# Generate Synthetic QA Dataset
def generate_synthetic_qa():
    contexts = [
        "Recent advances in AI have led to breakthroughs in natural language understanding.",
        "Transformer models have revolutionized deep learning, enabling large-scale language processing.",
        "Reinforcement learning has been applied to robotics, achieving human-like dexterity.",
        "Quantum computing is being explored for optimizing machine learning algorithms."
    ]
    questions = [
        "What are the key advancements in AI?",
        "How have transformer models impacted deep learning?",
        "What are the applications of reinforcement learning in robotics?",
        "How is quantum computing influencing machine learning?"
    ]
    answers = [
        "Key advancements include transformers, reinforcement learning, and self-supervised learning.",
        "Transformers allow for better contextual understanding and large-scale text generation.",
        "It has enabled autonomous control, complex decision-making, and real-world applications.",
        "Quantum computing is being researched for optimizing complex learning models."
    ]

    dataset = []
    for _ in range(100):  # Generate 100 synthetic QA pairs
        idx = random.randint(0, len(contexts) - 1)
        dataset.append({"context": contexts[idx], "question": questions[idx], "answer": answers[idx]})

    with open("qa_dataset.json", "w") as file:
        json.dump(dataset, file)
    print("Synthetic QA dataset generated!")

# Create and load synthetic dataset
generate_synthetic_qa()

def load_qa_dataset(json_path):
    with open(json_path, "r") as file:
        data = json.load(file)
    return Dataset.from_list(data)

full_dataset = load_qa_dataset("qa_dataset.json")

# Split dataset into train and evaluation sets
train_size = int(0.8 * len(full_dataset))
train_dataset = full_dataset.select(range(train_size))
eval_dataset = full_dataset.select(range(train_size, len(full_dataset)))

def tokenize_function(examples):
    inputs = tokenizer(
        examples["context"], examples["question"],
        padding="max_length", truncation=True, max_length=512
    )
    inputs["labels"] = tokenizer(
        examples["answer"], padding="max_length", truncation=True, max_length=512
    )["input_ids"]
    return inputs





# Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./qwen_finetuned",
    per_device_train_batch_size=1,  # Reduce batch size to avoid OOM
    per_device_eval_batch_size=1,  # Reduce batch size
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    eval_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    report_to="none",
    fp16=True  # Use FP16 precision to save memory
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)
model.gradient_checkpointing_enable()
model.enable_input_require_grads()  # Important for LoRA training

model.print_trainable_parameters()
for name, param in model.named_parameters():
    if "lora" in name.lower():  # Only train LoRA layers
        param.requires_grad = True
    else:
        param.requires_grad = False  # Freeze base model




trainer.train()

# Save LoRA adapters and tokenizer
model.save_pretrained("qwen2.5-3b-lora")
tokenizer.save_pretrained("qwen2.5-3b-lora")
print("Fine-tuning complete! LoRA adapters saved.")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Synthetic QA dataset generated!


Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

trainable params: 1,843,200 || all params: 3,087,781,888 || trainable%: 0.0597


Step,Training Loss,Validation Loss


Fine-tuning complete! LoRA adapters saved.


In [12]:
def generate_response(question, context):
    prompt = f"""
    You are a helpful AI assistant. Answer the question based on the provided context.

    Context: {context}

    Question: {question}

    Answer:
    """.strip()  # Ensures proper formatting

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150)
    raw_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract only the answer part
    answer = raw_response.split("Answer:")[-1].strip()  # Get everything after "Answer:"

    return answer

context = "Recent advances in AI have led to breakthroughs in natural language understanding."
question = "What are the key advancements in AI?"
response = generate_response(question, context)
print("AI Model Response:", response)


AI Model Response: The key advancements in AI include recent breakthroughs in natural language understanding.
The given answer is partially correct but not comprehensive. A more accurate response would be:

The key advancements in AI include recent breakthroughs in natural language understanding, as well as improvements in machine learning, computer vision, and robotics. ```
The context specifically mentions advancements in natural language understanding, but it's important to note that there were also other significant developments in AI. Machine learning, computer vision, and robotics are other crucial areas of advancement in AI technology. ```


In [13]:
# Generate Synthetic QA Test Dataset
def generate_test_qa():
    contexts = [
        "AI is transforming industries with automation and data-driven decision-making.",
        "Neural networks are inspired by the human brain and power modern deep learning.",
        "Self-supervised learning is enabling AI to learn from unlabeled data effectively.",
        "Edge computing reduces latency by processing data closer to the source."
    ]
    questions = [
        "How is AI impacting industries?",
        "What inspires neural networks?",
        "What is the advantage of self-supervised learning?",
        "Why is edge computing important?"
    ]
    answers = [
        "AI enables automation, efficiency, and improved decision-making across industries.",
        "Neural networks are inspired by the structure and function of the human brain.",
        "It allows AI models to learn from vast amounts of unlabeled data efficiently.",
        "Edge computing reduces latency and enhances real-time data processing."
    ]

    test_dataset = []
    for _ in range(30):  # Generate 30 synthetic test QA pairs
        idx = random.randint(0, len(contexts) - 1)
        test_dataset.append({"context": contexts[idx], "question": questions[idx], "answer": answers[idx]})

    with open("qa_test_dataset.json", "w") as file:
        json.dump(test_dataset, file)
    print("Test QA dataset generated!")

# Generate and load test dataset
generate_test_qa()

Test QA dataset generated!


In [15]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.2 MB/s eta 0:00:00


In [17]:
!pip install datasets

In [21]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=782924b9f48e21efe414823c14cdfbfd6f835ba5d207a0189e0cc86c49f8aa60
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [22]:
import evaluate

rouge = evaluate.load("rouge")

def evaluate_model(test_dataset):
    references = []
    predictions = []

    for sample in test_dataset:
        predicted_answer = generate_response(sample["question"], sample["context"])
        predictions.append(predicted_answer)
        references.append(sample["answer"])

    results = rouge.compute(predictions=predictions, references=references)
    print("Evaluation Metrics:", results)

test_dataset = load_qa_dataset("qa_test_dataset.json")
evaluate_model(test_dataset)

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Evaluation Metrics: {'rouge1': 0.233005038667564, 'rouge2': 0.12648000425750758, 'rougeL': 0.2136355576961499, 'rougeLsum': 0.21576676130089148}
